<a href="https://colab.research.google.com/github/Marcin19721205/BDCaseStudy/blob/main/BDCSZad4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Zaczynam od prostego założenia: chcę przewidzieć, kto kupi produkt (Purchase), ale bez „oszukiwania” danymi, które zdradzają odpowiedź. W tym zbiorze jest kolumna Spending, która w praktyce mówi  czy zakup nastąpił (prawie zawsze Spending > 0 oznacza Purchase = 1 - jak w uwadze do zadania - to spawdzone w kodzie). To klasyczny data leakage, więc Spending pomijam — inaczej model wyglądałby świetnie tylko dlatego, że podgląda wynik.

2. Model buduję jako segmentację klientów na podstawie dwóch cech, zgodnie z zadaniem: Recency i Frequency. Recency biorę z last_update_days_ago (ile dni minęło od zakupu), a Frequency z Freq (częstotkliwosc zakupow). Obie zmienne zamieniam na kategorie: last_update_days_ago dzielę na 4 koszyki (kwartyle wyliczone na zbiorze treningowym), a Freq na 3 koszyki (tercyle również z treningu). Z tych dwóch kodów tworzę kod segmentu RFcode (przykladow. „13” = Rcode=1 i Fcode=3).

3. Dla każdego segmentu na zbiorze TRAIN liczę rr = mean(Purchase), czyli odsetek klientów, którzy kupili. To jest cała inteligencja modelu: klient dostaje wynik score_rr równy rr swojego segmentu. Na zbiorze testowym nie liczę nic od nowa — tylko mapuję klienta do segmentu i przypisuję mu score_rr wyuczone na treningu.

Dzięki temu mam prosty, przejrzysty model segmentacyjny wejsciem są last_update_days_ago i Freq (po binowaniu), wyjściem jest Purchase, a predykcją jest prawdopodobieństwo zakupu oszacowane jako historyczny response rate segmentu.

In [2]:
import pandas as pd
import numpy as np

In [4]:
path = "sample_data/Tayko.csv"  # plik z zadania
df = pd.read_csv(path)  # load
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   sequence_number       2000 non-null   int64
 1   US                    2000 non-null   int64
 2   source_a              2000 non-null   int64
 3   source_c              2000 non-null   int64
 4   source_b              2000 non-null   int64
 5   source_d              2000 non-null   int64
 6   source_e              2000 non-null   int64
 7   source_m              2000 non-null   int64
 8   source_o              2000 non-null   int64
 9   source_h              2000 non-null   int64
 10  source_r              2000 non-null   int64
 11  source_s              2000 non-null   int64
 12  source_t              2000 non-null   int64
 13  source_u              2000 non-null   int64
 14  source_p              2000 non-null   int64
 15  source_x              2000 non-null   int64
 16  source

In [6]:
df.head()

,sequence_number,US,source_a,source_c,source_b,source_d,source_e,source_m,source_o,source_h,...,source_x,source_w,Freq,last_update_days_ago,1st_update_days_ago,Web order,Gender=male,Address_is_res,Purchase,Spending
0,1,1,0,0,1,0,0,0,0,0,...,0,0,2,3662,3662,1,0,1,1,128
1,2,1,0,0,0,0,1,0,0,0,...,0,0,0,2900,2900,1,1,0,0,0
2,3,1,0,0,0,0,0,0,0,0,...,0,0,2,3883,3914,0,0,0,1,127
3,4,1,0,1,0,0,0,0,0,0,...,0,0,1,829,829,0,1,0,0,0
4,5,1,0,1,0,0,0,0,0,0,...,0,0,1,869,869,0,0,0,0,0


In [14]:
#shape, typy, braki, duplikaty, target Purchase)
shape = df.shape  # (N, p)
dtypes = df.dtypes.value_counts().to_dict()  # typy
na_total = int(df.isna().sum().sum())  # braki ogółem
na_by_col = df.isna().sum().sort_values(ascending=False)  # braki per kolumna
dup_rows = int(df.duplicated().sum())  # duplikaty wierszy

purchase_dtype = df["Purchase"].dtype  # typ targetu
purchase_unique = df["Purchase"].dropna().unique()  # unikalne wartości (bez NaN)
purchase_counts = df["Purchase"].value_counts(dropna=False).to_dict()  # rozkład klas
purchase_rate = float(df["Purchase"].mean())  # odsetek 1 (jeśli 0/1)

print("shape",shape)
print("dtypes", dtypes)
print("na_total",na_total)
print("na_by_col",na_by_col)
print("dup_rows",dup_rows)
print("purchase_dtype",purchase_dtype)
print("purchase_unique",purchase_unique)
print("purchase_counts",purchase_counts)
print("purchase_rate",purchase_rate)

shape (2000, 25)
dtypes {dtype('int64'): 25}
na_total 0
na_by_col sequence_number         0
US                      0
source_a                0
source_c                0
source_b                0
source_d                0
source_e                0
source_m                0
source_o                0
source_h                0
source_r                0
source_s                0
source_t                0
source_u                0
source_p                0
source_x                0
source_w                0
Freq                    0
last_update_days_ago    0
1st_update_days_ago     0
Web order               0
Gender=male             0
Address_is_res          0
Purchase                0
Spending                0
dtype: int64
dup_rows 0
purchase_dtype int64
purchase_unique [1 0]
purchase_counts {1: 1000, 0: 1000}
purchase_rate 0.5


Niezbalansowanie klas

In [15]:
#imbalance klas dla Purchase (liczności + %)
purchase_counts = df["Purchase"].value_counts(dropna=False).sort_index()  # liczności
purchase_pct = (df["Purchase"].value_counts(normalize=True, dropna=False).sort_index() * 100).round(2)  # %

pd.DataFrame({"count": purchase_counts, "pct": purchase_pct})
#jest super ok

,count,pct
Purchase,,
0,1000,50.0
1,1000,50.0


Walidacja spending

sprawdzam cz spending jest leakiem



1. sprawdzam czy w danych są kolumny Spending i Purchase


2. Biore tylko wiersze, gdzie obie wartości nie są NaN.

3. tworz sp_gt0 = 1, jeśli Spending > 0, else 0.

4. tabela krzyżowa ct między sp_gt0 i Purchase

5. mismatch: ile wierszy ma inną wartość sp_gt0 then Purchase.

6. mismatch_rate: odsetek tych błędów.

--- Ananliza---------

ct pokazuje prawie idealną regułę:

sp_gt0=0 -> Purchase=0: 999

sp_gt0=1 -> Purchase=1: 1000

a. wyjątki = 1 przypadek sp_gt0=1 i Purchase=0 co daje mismatch=1, mismatch_rate=0.0005 (0,05%)

spending praktycznie determinuje purchase, więc to data leakage i wyrzucam Spending z predyktorów

In [20]:
# falidacja leakage: Spending>0 <==> Purchase=1 (tabela zgodności + % błędów)
has_spending = "Spending" in df.columns  # czy jest kolumna
has_purchase = "Purchase" in df.columns  # czy jest kolumna

chk = df.loc[df["Spending"].notna() & df["Purchase"].notna(), ["Spending", "Purchase"]].copy()  # tylko kompletne
chk["sp_gt0"] = (chk["Spending"] > 0).astype(int)  # Spending>0 -> 1/0

ct = pd.crosstab(chk["sp_gt0"], chk["Purchase"], dropna=False)  # zgodność
mismatch = int((chk["sp_gt0"] != chk["Purchase"].astype(int)).sum())  # ile nie pasuje
mismatch_rate = mismatch / len(chk) if len(chk) else np.nan  # % niezgodności

print("has spending", has_spending)
print("has purchase", has_purchase)
print("ct", ct)
print("mismatch", mismatch)
print("mismatch rate", mismatch_rate)



has spending True
has purchase True
ct Purchase    0     1
sp_gt0             
0         999     0
1           1  1000
mismatch 1
mismatch rate 0.0005


In [29]:
#cechy segmentacji - wybrane (Recency+Frequency) - jak w treści uwag do zadania + kontrola kolumn
seg_cols = ["last_update_days_ago", "Freq"]  # Recency + Frequency
need_cols = seg_cols + ["Purchase"]  # do rr(Purchase) w segmentach

missing = [c for c in need_cols if c not in df.columns]  # braki

print("\ncechy segmentacji = Recency+Frequency")
print("X:", seg_cols, "| y: Purchase")
print("Brakujące kolumny:", missing if missing else "brak")

if not missing:
    dtypes_seg = df[seg_cols].dtypes.to_dict()  # typy X
    na_seg = df[need_cols].isna().sum().to_dict()  # braki X+y

    print("Typy:", {k: str(v) for k, v in dtypes_seg.items()})
    print("NaN:", {k: int(v) for k, v in na_seg.items()})





cechy segmentacji = Recency+Frequency
X: ['last_update_days_ago', 'Freq'] | y: Purchase
Brakujące kolumny: brak
Typy: {'last_update_days_ago': 'int64', 'Freq': 'int64'}
NaN: {'last_update_days_ago': 0, 'Freq': 0, 'Purchase': 0}


In [36]:
#koszyki (train) + te same progi na test, kod segmentu = RcodeFcode
import numpy as np
import pandas as pd

# foreword: Zad.4 — split 60/40 (wymagane przed krokami z train_df/test_df)
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.4, random_state=123, shuffle=True)
train_df.shape, test_df.shape


# Recency (last_update_days_ago) -> 4 koszyki (kwartyle) na TRAIN
train_df["Rcode"], r_bins = pd.qcut(
    train_df["last_update_days_ago"], q=4, labels=[1, 2, 3, 4], retbins=True, duplicates="drop"
)

# Frequency (Freq) -> 3 koszyki (tercyle) na TRAIN
train_df["Fcode"], f_bins = pd.qcut(
    train_df["Freq"], q=3, labels=[1, 2, 3], retbins=True, duplicates="drop"
)

train_df["Rcode"] = train_df["Rcode"].astype(int)  # kody int
train_df["Fcode"] = train_df["Fcode"].astype(int)  # kody int
train_df["RFcode"] = train_df["Rcode"].astype(str) + train_df["Fcode"].astype(str)  # kod segmentu

# TEST: te same progi co TRAIN
#biny z train rozszerzone do [-inf, inf] + kody jako Int64 zakomentowane wywalały błąd
r_bins2 = r_bins.copy()  # copy
r_bins2[0] = -np.inf  # lewy skraj
r_bins2[-1] = np.inf  # prawy skraj

f_bins2 = f_bins.copy()  # copy
f_bins2[0] = -np.inf  # lewy skraj
f_bins2[-1] = np.inf  # prawy skraj

test_df["Rcode"] = pd.cut(test_df["last_update_days_ago"], bins=r_bins2, labels=[1, 2, 3, 4], include_lowest=True).astype("Int64")
test_df["Fcode"] = pd.cut(test_df["Freq"], bins=f_bins2, labels=[1, 2, 3], include_lowest=True).astype("Int64")
test_df["RFcode"] = test_df["Rcode"].astype(str) + test_df["Fcode"].astype(str)

test_df[["last_update_days_ago", "Freq", "Rcode", "Fcode", "RFcode"]].isna().sum()

#wywalały błąd
#test_df["Rcode"] = pd.cut(test_df["last_update_days_ago"], bins=r_bins, labels=[1, 2, 3, 4], include_lowest=True).astype(int)
#test_df["Fcode"] = pd.cut(test_df["Freq"], bins=f_bins, labels=[1, 2, 3], include_lowest=True).astype(int)
#test_df["RFcode"] = test_df["Rcode"].astype(str) + test_df["Fcode"].astype(str)

print("\nzrobione koszyki i kod segmentu RF")
print("Progi R na train:", np.round(r_bins, 3))
print("Progi F na train:", np.round(f_bins, 3))
print("Liczność segmentów -Train:", train_df["RFcode"].value_counts().sort_index().to_dict())
print("Liczność segmentów -TESt: ", test_df["RFcode"].value_counts().sort_index().to_dict())



zrobione koszyki i kod segmentu RF
Progi R na train: [   7.   1043.75 2140.   3048.25 4188.  ]
Progi F na train: [ 0.  1.  2. 15.]
Liczność segmentów -Train: {'11': 174, '12': 52, '13': 74, '21': 128, '22': 114, '23': 58, '31': 239, '32': 39, '33': 22, '41': 234, '42': 56, '43': 10}
Liczność segmentów -TESt:  {'11': 87, '12': 27, '13': 31, '21': 95, '22': 71, '23': 41, '31': 175, '32': 28, '33': 15, '41': 187, '42': 32, '43': 11}


In [38]:
# rr=mean(Purchase) na TRAIN per RFcode + score_rr na TRAIN/TEST
rr_seg = (train_df.groupby("RFcode")["Purchase"]
          .agg(customers="size", purchases="sum", rr="mean")
          .reset_index()
          .sort_values("rr", ascending=False)
          .reset_index(drop=True))  # tabela segmentów

rr_map = rr_seg.set_index("RFcode")["rr"]  # RFcode -> rr (TRAIN)

train_df["score_rr"] = train_df["RFcode"].map(rr_map)  # score na TRAIN
test_df["score_rr"] = test_df["RFcode"].map(rr_map)    # score na TEST

print("\nrr(segment)=mean(Purchase) na TRAIN + score_rr przypisany przez mapowanie.")
print("Top segmenty wg rr:")
print(rr_seg.head(10).to_string(index=False))
print("Braki score_rr (TRAIN/TEST):", int(train_df["score_rr"].isna().sum()), int(test_df["score_rr"].isna().sum()))



rr(segment)=mean(Purchase) na TRAIN + score_rr przypisany przez mapowanie.
Top segmenty wg rr:
RFcode  customers  purchases       rr
    33         22         22 1.000000
    43         10         10 1.000000
    13         74         70 0.945946
    23         58         54 0.931034
    32         39         36 0.923077
    42         56         42 0.750000
    12         52         38 0.730769
    22        114         57 0.500000
    31        239         97 0.405858
    21        128         45 0.351562
Braki score_rr (TRAIN/TEST): 0 0


In [40]:
# najbardziej obiecujący segment: max rr przy sensownej liczności + opis (R/F zakresy, n, rr)
min_n = 30  # minimum klientów w segmencie

rr_ok = rr_seg.loc[rr_seg["customers"] >= min_n].copy()  # filtr liczności
best_seg = rr_ok.iloc[0].copy() if len(rr_ok) else rr_seg.iloc[0].copy()  # fallback gdy brak segmentów >= min_n

best_rf = best_seg["RFcode"]  # kod segmentu
best_n = int(best_seg["customers"])  # liczność
best_rr = float(best_seg["rr"])  # response rate

r_code = int(str(best_rf)[0])  # Rcode
f_code = int(str(best_rf)[1])  # Fcode

# zakresy (z binów na TRAIN) dla R i F
r_lo, r_hi = float(r_bins2[r_code-1]), float(r_bins2[r_code])  # przedział Recency
f_lo, f_hi = float(f_bins2[f_code-1]), float(f_bins2[f_code])  # przedział Frequency

print("\nnajbardziej obiecujący segment (max rr + filtr liczności).")
print(f"Segment RFcode: {best_rf} | n={best_n} | rr={best_rr:.4f}")
print(f"Recency (last_update_days_ago) bin Rcode={r_code}: [{r_lo:.3f}, {r_hi:.3f}]")
print(f"Frequency (Freq) bin Fcode={f_code}: [{f_lo:.3f}, {f_hi:.3f}]")
print(f"filtr liczności min_n={min_n} | użyto_filtrowania={bool(len(rr_ok))}")



najbardziej obiecujący segment (max rr + filtr liczności).
Segment RFcode: 13 | n=74 | rr=0.9459
Recency (last_update_days_ago) bin Rcode=1: [-inf, 1043.750]
Frequency (Freq) bin Fcode=3: [2.000, inf]
filtr liczności min_n=30 | użyto_filtrowania=True
